In [1]:
from sentence_transformers import SentenceTransformer
from lenses import lens

import torch

import rdflib

cos = torch.nn.CosineSimilarity()

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b, field="subject"):
    embeddings = model.encode([a[field],
                               b[field]])
    return dot(embeddings[0], embeddings[1])/(norm(embeddings[0])*norm(embeddings[1]))

# Modelado de datos

### La información de Confluence debiera extraerse via API.

El siguiente código sólo cumple una función de ejemplo de cómo se estructura un documento de confluence

El html es material confidencial, asi que no se adjunta.

In [2]:
from bs4 import BeautifulSoup

listfiles = ["PPM100110310 - 1. DR Requerimientos - Convenios productos activo - Confluence.html"]

with open(listfiles[0], encoding="utf8") as f:
    soup = BeautifulSoup(f, 'html.parser')

In [3]:
document = {}

for title in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):

    title_text = title.get_text()
    
    content = ""
    next_item = title.find_next_sibling()
    
    while next_item and next_item.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
        content += next_item.get_text() + ' '
        next_item = next_item.find_next_sibling()

    document[title_text] = content.strip()

In [4]:
document.keys()

dict_keys(['Accesos directos de espacio', 'Árbol de páginas', '\nPPM100110310 - 1. DR Requerimientos\n', 'Introducción', 'Versiones documento', 'Objetivo del documento', 'Contexto', 'Alcance y objetivos del proyecto', 'Intervinientes', 'Inventario de requerimientos', 'Detalle y Descripción de Requerimientos\xa0', '\xa0', 'Fuera de Alcance', 'Información Adicional', 'Planificación', 'Tareas'])

### Ejemplo json

Las páginas de confluence debieran parsearse con el fin de obtener un objeto json almacenable en BBDD

In [5]:
projects = {"encuestas_a_empleados": {"subject": "En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda.",
                                        "url": "..."},
             "Nuevo_acuerdo_FEI_INVESTEU": {"subject": "Ineluctable modality of the visible: at least that if no more, thought through my eyes. Signatures of all things I am here to read, seaspawn and seawrack, the nearing tide, that rusty boot. Snotgreen, bluesilver, rust: coloured signs. Limits of the diaphane. But he adds: in bodies. Then he was aware of them bodies before of them coloured. How? By knocking his sconce against them, sure. Go easy. Bald he was and a millionaire, maestro di color che sanno. Limit of the diaphane in. Why in? Diaphane, adiaphane. If you can put your five fingers through it it is a gate, if not a door. Shut your eyes and see.",
                                           "url": "..."},
             "FEI_InvestEU_Sostenibilidad": {"subject": "It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair, we had everything before us, we had nothing before us, we were all going direct to Heaven, we were all going direct the other way—in short, the period was so far like the present period, that some of its noisiest authorities insisted on its being received, for good or for evil, in the superlative degree of comparison only.",
                                            "url": "..."},
             "FEI_InvestEU_Innovfin": {"subject": "Pues sepa V.M. ante todas cosas que a mí llaman Lázaro de Tormes, hijo de Tomé González y de Antona Pérez, naturales de Tejares, aldea de Salamanca. Mi nacimiento fue dentro del río Tormes, por la cual causa tomé el sobrenombre, y fue desta manera. Mi padre, que Dios perdone, tenía cargo de proveer una molienda de una aceña, que está ribera de aquel río, en la cual fue molinero más de quince años; y estando mi madre una noche en la aceña, preñada de mí, tomóle el parto y parióme allí: de manera que con verdad puedo decir nacido en el río.",
                                       "url": "..."}
           }

# Red semántica

Los csv de ejemplo son material confidencial. No se adjuntan.

EL almacenamiento elegido es Neo4J.

In [6]:
import pandas as pd

df = pd.read_csv("Maestro PGS - Backlog - Servicios IT2.csv", sep=';')

df.dropna(axis=0, how='all', inplace=True)

def get_index(columns, col_name):
    return list(columns).index(col_name) + 1

from functools import partial
gip = partial(get_index, df.columns)

dicto_MaestroPGS = {}

for row in df.itertuples():
    dicto_MaestroPGS[row[gip('ID Servicio IT')]] = {'url': row[gip('Id Project Group')],
                                                    'Responsable': row[gip('Resp. Servicio IT')],
                                                     'Matricula_responsable': row[gip('Matricula\xa0\n(Resp. Servicio IT)')],
                                                     'Grupo': row[gip('Grupo')],
                                                     'Grupo_id': row[gip('ID Grupo')],
                                                     'Grupo_responsable': row[gip('Resp. Grupo')],
                                                     'Grupo_matricula_responsable': row[gip('Matricula\xa0\n(Resp. Grupo)')],
                                                     'Plataforma': row[gip('Plataforma')],
                                                     'Plataforma_ID': row[gip('ID Plataforma')],
                                                     'Plataforma_responsable': row[gip('Resp. Plataforma')],
                                                     'Plataforma_matricula_responsable': row[gip('Matricula\n(Resp. Plataforma)')]
                                                    }

In [7]:
df = pd.read_csv("Relacion_PGS_PPM.csv", sep=';')
df.dropna(axis=0, how='all', inplace=True)

gip = partial(get_index, df.columns)

dicto_Relacion_PGS_PGS = {}

for row in df.itertuples():
    dicto_Relacion_PGS_PGS[row[gip('ID del proyecto')]] = {'url': row[gip('Link')],
                                                    'Tipologia': row[gip('Tipo')],
                                                    'Nombre_del_proyecto': row[gip('Nombre del proyecto')],
                                                    'Estado': row[gip('Estado')],
                                                    'Peticionario': row[gip('Peticionario (usuario)')],
                                                    'Peticionario_Grupo': row[gip('Grupo peticionario')],
                                                    'Gestor': row[gip('Gestor')],
                                                    'Gestor_Grupo': row[gip('Grupo gestor')],
                                                    'Ejercicio': None if np.isnan(row[gip('Ejercicio')]) else int(row[gip('Ejercicio')]),
                                                    'Plataforma_gestora': row[gip('Plataforma gestora')]}

In [8]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF
from rdflib.plugins.sparql import prepareQuery

class RDF_graph:
    def __init__(self):
        self.FOAF = Namespace("http://xmlns.com/foaf/0.1/")
        self.ORG = Namespace("http://example.org/organization#")
        self.PROJECT = Namespace("http://example.org/project#")
        
        self.g = Graph()
        
    def add_node_rdf(self, identifier, **kwargs):
        identifier = URIRef(f"http://example.org/service#{identifier}")
        url = URIRef(kwargs['url'])
        Responsable = Literal(kwargs['Responsable'])
        Matricula_responsable =  f"https://pplnow.colabora.lacaixa.es/SitePages/Profile.aspx?IdUser={kwargs['Matricula_responsable']}"
        Grupo_id = Literal(kwargs['Grupo_id'])
        caixa = URIRef("http://example.org/organization/caixa")
        chatbot = URIRef("http://example.org/project/chatbot")
        migracion_cloud = URIRef("http://example.org/project/migracion-cloud")

        
        self.g.add((Responsable, RDF.type, self.FOAF.Person))
        self.g.add((Responsable, self.ORG.isResponsibleFor, identifier))
        self.g.add((identifier, self.ORG.belongsTo, caixa))
        self.g.add((identifier, self.ORG.hasLeader, Responsable))
        self.g.add((identifier, self.ORG.isRelatedTo, chatbot))
        self.g.add((identifier, self.ORG.dependsOn, migracion_cloud))
        self.g.add((identifier, self.ORG.relationshipStrength, Literal("50%")))
    
    def query(self, query):
        results = self.g.query(query)
        
        return results
    
    def export_ttl(self, filepath):
        ontology_ttl = self.g.serialize(format='turtle')

        with open(filepath, "w") as f:
            f.write(ontology_ttl)
        

In [9]:
dicto_MaestroPGS['llm'] = {'url': "http://example.org/project#llm",
  'Responsable': 'Smithee, Alan',
  'Matricula_responsable': '666',
  'Grupo': 'IA',
  'Grupo_id': 'GBGR00269',
  'Grupo_responsable': 'Bourbaki, Nicolas',
  'Grupo_matricula_responsable': '777',
  'Plataforma': 'Products, Clients & Accounts',
  'Plataforma_ID': 'GBPL00045',
  'Plataforma_responsable': 'Eco, Umberto',
  'Plataforma_matricula_responsable': '888'}

In [10]:
g = RDF_graph()

g.add_node_rdf(identifier='llm', **dicto_MaestroPGS['llm'])

In [11]:
print(g.g.serialize())

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ns1: <http://example.org/organization#> .

<http://example.org/service#llm> ns1:belongsTo <http://example.org/organization/caixa> ;
    ns1:dependsOn <http://example.org/project/migracion-cloud> ;
    ns1:hasLeader "Smithee, Alan" ;
    ns1:isRelatedTo <http://example.org/project/chatbot> ;
    ns1:relationshipStrength "50%" .

"Smithee, Alan" a foaf:Person ;
    ns1:isResponsibleFor <http://example.org/service#llm> .




In [12]:
query = prepareQuery(
    """
    PREFIX ns1: <http://example.org/organization#>
    PREFIX service: <http://example.org/service#>

    SELECT ?dependency
    WHERE {
        service:llm ns1:dependsOn ?dependency.
    }
    """
)

results = g.query(query)

In [13]:
[row for row in results]

[(rdflib.term.URIRef('http://example.org/project/migracion-cloud'),)]

# Vector similarity search

Sirva este adendo de ejemplo de cómo funcionaría la búsqueda semántica, no obstante se usaría Weviate.

In [14]:
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [15]:
cos_sim(projects["encuestas_a_empleados"], projects["Nuevo_acuerdo_FEI_INVESTEU"])

0.30116013

In [16]:
cos_sim(projects["FEI_InvestEU_Sostenibilidad"], projects["FEI_InvestEU_Innovfin"])

0.13825643

In [17]:
cos_sim(projects["encuestas_a_empleados"], projects["FEI_InvestEU_Innovfin"])

0.65123504